## **Install Spark**

In [1]:
!apt-get install openjdk-8-jdk-headless -qq
!pip install -q findspark
!pip install pyspark==3.4.1

Selecting previously unselected package libxtst6:amd64.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u472-ga-1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u472-ga-1~22.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u472-ga-1~22.04_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u472-ga-1~22.04) ...
Setting up libxtst6:amd64 (2:1.2.3-1build4) ...
Setting up openjdk-8-jre-headless:amd64 (8u472-ga-1~22.04) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/jjs to provide /usr/bin/jjs (jjs) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/pack200 to provide /usr/bin/pack200 (pack20

In [2]:
!rm -rf /content/spark-3.5.0-bin-hadoop3*


In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar -xvf spark-3.4.1-bin-hadoop3.tgz


spark-3.4.1-bin-hadoop3/
spark-3.4.1-bin-hadoop3/R/
spark-3.4.1-bin-hadoop3/R/lib/
spark-3.4.1-bin-hadoop3/R/lib/sparkr.zip
spark-3.4.1-bin-hadoop3/R/lib/SparkR/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/R.css
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/00Index.html
spark-3.4.1-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/aliases.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/AnIndex
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdx
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdb
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/paths.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/worker.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/daemon.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.4.1-bin-hadoop3/R/lib/S

In [4]:
!ls /content/spark-3.4.1-bin-hadoop3/bin

beeline		      pyspark		spark-class.cmd      spark-shell.cmd
beeline.cmd	      pyspark2.cmd	spark-connect-shell  spark-sql
docker-image-tool.sh  pyspark.cmd	sparkR		     spark-sql2.cmd
find-spark-home       run-example	sparkR2.cmd	     spark-sql.cmd
find-spark-home.cmd   run-example.cmd	sparkR.cmd	     spark-submit
load-spark-env.cmd    spark-class	spark-shell	     spark-submit2.cmd
load-spark-env.sh     spark-class2.cmd	spark-shell2.cmd     spark-submit.cmd


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("BankingDataAnalysis") \
    .getOrCreate()

spark

In [6]:
spark.version

'3.4.1'

**Load Dataset**

In [7]:
df = spark.read.csv("/content/bank.csv", header=True, inferSchema=True)
df.show(5)
df.printSchema()


+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|        job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30| unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|   services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35| management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
| 30| management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown| no|
| 59|blue-collar|married|secondary|     no|      0|    yes|  no| unknown|  5|  may|     22

**Basic EDA**

In [8]:
df.count()

4521

In [9]:
df.describe(["age", "balance", "duration"]).show()

+-------+------------------+------------------+------------------+
|summary|               age|           balance|          duration|
+-------+------------------+------------------+------------------+
|  count|              4521|              4521|              4521|
|   mean| 41.17009511170095|1422.6578190665782|263.96129174961294|
| stddev|10.576210958711263|3009.6381424673395|259.85663262468216|
|    min|                19|             -3313|                 4|
|    max|                87|             71188|              3025|
+-------+------------------+------------------+------------------+



In [10]:
df.filter(df.balance > 1000).show(5)

+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|       job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|  services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35|management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
| 30|management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown| no|
| 39|  services|married|secondary|     no|   9374|    yes|  no| unknown| 20|  may|     273|      

**Create Quarter Column**

In [11]:
from pyspark.sql.functions import when

df = df.withColumn(
    "quarter",
    when(df.month.isin("jan","feb","mar"), "Q1")
    .when(df.month.isin("apr","may","jun"), "Q2")
    .when(df.month.isin("jul","aug","sep"), "Q3")
    .otherwise("Q4")
)

In [12]:
df.select("month", "quarter").show(5)


+-----+-------+
|month|quarter|
+-----+-------+
|  oct|     Q4|
|  may|     Q2|
|  apr|     Q2|
|  jun|     Q2|
|  may|     Q2|
+-----+-------+
only showing top 5 rows



**GROUP BY & AGGREGATION**

In [13]:
df.groupBy("job").avg("balance").show()

+-------------+------------------+
|          job|      avg(balance)|
+-------------+------------------+
|   management|1766.9287925696594|
|      retired| 2319.191304347826|
|      unknown|1501.7105263157894|
|self-employed|1392.4098360655737|
|      student|1543.8214285714287|
|  blue-collar| 1085.161733615222|
| entrepreneur|          1645.125|
|       admin.|  1226.73640167364|
|   technician|     1330.99609375|
|     services|1103.9568345323742|
|    housemaid|2083.8035714285716|
|   unemployed|       1089.421875|
+-------------+------------------+



In [14]:
df.groupBy("marital").count().show()

+--------+-----+
| marital|count|
+--------+-----+
|divorced|  528|
| married| 2797|
|  single| 1196|
+--------+-----+



**UDF — AGE GROUPING**

In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def age_group(age):
    if age < 30:
        return "<30"
    elif age <= 60:
        return "30-60"
    else:
        return ">60"

age_udf = udf(age_group, StringType())
df = df.withColumn("age_group", age_udf(df.age))


In [16]:
df.select("age", "age_group").show(5)


+---+---------+
|age|age_group|
+---+---------+
| 30|    30-60|
| 33|    30-60|
| 35|    30-60|
| 30|    30-60|
| 59|    30-60|
+---+---------+
only showing top 5 rows



**PARK SQL QUERIES**

In [17]:
df.createOrReplaceTempView("bank")


**Average balance by age group**

In [18]:
spark.sql("""
SELECT age_group, AVG(balance) AS avg_balance
FROM bank
GROUP BY age_group
""").show()


+---------+------------------+
|age_group|       avg_balance|
+---------+------------------+
|      <30|1058.0954356846473|
|      >60|2676.3543307086616|
|    30-60|1426.8755112474437|
+---------+------------------+



**Subscription rate by education**

In [19]:
spark.sql("""
SELECT education,
       SUM(CASE WHEN y='yes' THEN 1 ELSE 0 END)*100.0/COUNT(*) AS subscription_rate
FROM bank
GROUP BY education
""").show()

+---------+-----------------+
|education|subscription_rate|
+---------+-----------------+
|  unknown|10.16042780748663|
| tertiary|14.29629629629630|
|secondary|10.62445793581960|
|  primary| 9.43952802359882|
+---------+-----------------+



### **SPARK ML — BANK TERM DEPOSIT PREDICTION**

**Goal : Predict whether a client subscribes to a term deposit (y: yes/no).**

In [20]:
from pyspark.sql.functions import col

# Keep only useful columns
df_ml = df.select(
    "age","balance","duration","campaign","pdays","previous",
    "job","marital","education","housing","loan","contact","month","poutcome","y"
)

df_ml.show(5)

+---+-------+--------+--------+-----+--------+-----------+-------+---------+-------+----+--------+-----+--------+---+
|age|balance|duration|campaign|pdays|previous|        job|marital|education|housing|loan| contact|month|poutcome|  y|
+---+-------+--------+--------+-----+--------+-----------+-------+---------+-------+----+--------+-----+--------+---+
| 30|   1787|      79|       1|   -1|       0| unemployed|married|  primary|     no|  no|cellular|  oct| unknown| no|
| 33|   4789|     220|       1|  339|       4|   services|married|secondary|    yes| yes|cellular|  may| failure| no|
| 35|   1350|     185|       1|  330|       1| management| single| tertiary|    yes|  no|cellular|  apr| failure| no|
| 30|   1476|     199|       4|   -1|       0| management|married| tertiary|    yes| yes| unknown|  jun| unknown| no|
| 59|      0|     226|       1|   -1|       0|blue-collar|married|secondary|    yes|  no| unknown|  may| unknown| no|
+---+-------+--------+--------+-----+--------+----------

In [21]:
from pyspark.ml.feature import StringIndexer

In [22]:
categorical_cols = ["job","marital","education","housing","loan","contact","month","poutcome"]
indexers = [
    StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid="keep")
    for c in categorical_cols
]

label_indexer = StringIndexer(inputCol="y", outputCol="label")

**FEATURE VECTOR**

In [23]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[
        "age","balance","duration","campaign","pdays","previous"
    ] + [c+"_idx" for c in categorical_cols],
    outputCol="features"
)


**MODEL SELECTION (LOGISTIC REGRESSION)**

In [24]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

lr = LogisticRegression(featuresCol="features", labelCol="label")

pipeline = Pipeline(stages=indexers + [label_indexer, assembler, lr])

In [25]:
train_df, test_df = df_ml.randomSplit([0.8, 0.2], seed=42)

model = pipeline.fit(train_df)
predictions = model.transform(test_df)

predictions.select("label","prediction","probability").show(5)

+-----+----------+--------------------+
|label|prediction|         probability|
+-----+----------+--------------------+
|  1.0|       0.0|[0.92476515744769...|
|  0.0|       0.0|[0.81898779786198...|
|  0.0|       0.0|[0.97891825639656...|
|  0.0|       0.0|[0.79970553448899...|
|  0.0|       0.0|[0.96612476922676...|
+-----+----------+--------------------+
only showing top 5 rows



**MODEL EVALUATION**

In [26]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="label",
    metricName="areaUnderROC"
)

roc_auc = evaluator.evaluate(predictions)
roc_auc

0.8939016712454216

**HYPERPARAMETER TUNING**

In [27]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5]) \
    .build()

cv = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=3
)

cvModel = cv.fit(train_df)

**FEATURE IMPORTANCE / COEFFICIENTS**

In [28]:
best_lr = cvModel.bestModel.stages[-1]

list(zip(assembler.getInputCols(), best_lr.coefficients))

[('age', np.float64(0.0)),
 ('balance', np.float64(0.0)),
 ('duration', np.float64(0.003447860838033757)),
 ('campaign', np.float64(-0.013946208787717664)),
 ('pdays', np.float64(-3.742057579056771e-05)),
 ('previous', np.float64(0.0)),
 ('job_idx', np.float64(0.0)),
 ('marital_idx', np.float64(0.1981010858909297)),
 ('education_idx', np.float64(0.0)),
 ('housing_idx', np.float64(0.3888661866518585)),
 ('loan_idx', np.float64(-0.4293741723191457)),
 ('contact_idx', np.float64(-0.21926984951931697)),
 ('month_idx', np.float64(0.12878645774638425)),
 ('poutcome_idx', np.float64(0.634622476185977))]

A Logistic Regression model was developed using Spark ML to predict term deposit subscription. Categorical variables were encoded using StringIndexer, features were assembled using VectorAssembler, and the model was evaluated using ROC-AUC. Hyperparameter tuning with CrossValidator improved model robustness. Feature coefficients provided interpretability of customer behavior.

**SIMULATED STREAMING DATA**

In [29]:
from pyspark.sql.functions import current_timestamp

stream_df = df.withColumn("event_time", current_timestamp())
stream_df.show(5)

+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+-------+---------+--------------------+
|age|        job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|quarter|age_group|          event_time|
+---+-----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+-------+---------+--------------------+
| 30| unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|     Q4|    30-60|2026-01-27 12:29:...|
| 33|   services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|     Q2|    30-60|2026-01-27 12:29:...|
| 35| management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|     Q2|    30-60|2026-01-

**AGGREGATION**

In [30]:
from pyspark.sql.functions import avg

stream_df.groupBy("job") \
         .agg(avg("balance").alias("avg_balance")) \
         .show()


+-------------+------------------+
|          job|       avg_balance|
+-------------+------------------+
|   management|1766.9287925696594|
|      retired| 2319.191304347826|
|      unknown|1501.7105263157894|
|self-employed|1392.4098360655737|
|      student|1543.8214285714287|
|  blue-collar| 1085.161733615222|
| entrepreneur|          1645.125|
|       admin.|  1226.73640167364|
|   technician|     1330.99609375|
|     services|1103.9568345323742|
|    housemaid|2083.8035714285716|
|   unemployed|       1089.421875|
+-------------+------------------+



**WINDOW OPERATION**

In [31]:
from pyspark.sql.functions import window

stream_df.groupBy(
    window("event_time", "10 minutes"),
    "job"
).agg(
    avg("balance").alias("avg_balance")
).show(truncate=False)

+------------------------------------------+-------------+------------------+
|window                                    |job          |avg_balance       |
+------------------------------------------+-------------+------------------+
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|retired      |2319.191304347826 |
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|unknown      |1501.7105263157894|
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|self-employed|1392.4098360655737|
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|technician   |1330.99609375     |
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|blue-collar  |1085.161733615222 |
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|unemployed   |1089.421875       |
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|services     |1103.9568345323742|
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|admin.       |1226.73640167364  |
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|housemaid    |2083.8035714285716|
|{2026-01-27 12:30:00, 2026-01-27 12:40:00}|entrepreneur |1645.1

**LATE DATA**

Late and out-of-order data handling is implemented conceptually using watermarking. In a real-time production system, Spark’s withWatermark() would be applied to event_time columns to discard delayed records beyond a threshold.

In [32]:
!zip -r spark-3.4.1-bin-hadoop3.zip spark-3.4.1-bin-hadoop3

  adding: spark-3.4.1-bin-hadoop3/ (stored 0%)
  adding: spark-3.4.1-bin-hadoop3/python/ (stored 0%)
  adding: spark-3.4.1-bin-hadoop3/python/dist/ (stored 0%)
  adding: spark-3.4.1-bin-hadoop3/python/.coveragerc (deflated 41%)
  adding: spark-3.4.1-bin-hadoop3/python/setup.py (deflated 70%)
  adding: spark-3.4.1-bin-hadoop3/python/MANIFEST.in (deflated 48%)
  adding: spark-3.4.1-bin-hadoop3/python/docs/ (stored 0%)
  adding: spark-3.4.1-bin-hadoop3/python/docs/make.bat (deflated 44%)
  adding: spark-3.4.1-bin-hadoop3/python/docs/Makefile (deflated 45%)
  adding: spark-3.4.1-bin-hadoop3/python/docs/make2.bat (deflated 48%)
  adding: spark-3.4.1-bin-hadoop3/python/docs/source/ (stored 0%)
  adding: spark-3.4.1-bin-hadoop3/python/docs/source/_static/ (stored 0%)
  adding: spark-3.4.1-bin-hadoop3/python/docs/source/_static/css/ (stored 0%)
  adding: spark-3.4.1-bin-hadoop3/python/docs/source/_static/css/pyspark.css (deflated 55%)
  adding: spark-3.4.1-bin-hadoop3/python/docs/source/_stati

In [33]:
from google.colab import files
files.download("spark-3.4.1-bin-hadoop3.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>